In [1]:
import pandas as pd
import os
import re
import warnings
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import time

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [ ]:
def leitura_bases(dados_ssp_sp, dict_bases, regioes=None):
    """
    Carregar os arquivos baixados a partir do script 'web_scrapping_bases.py'.
    Esse serão organizados em um dicionário segmentado por ano, região e município.
    
    Args:
        dados_ssp_sp O nome da pasta criada no "web_scrapping_bases.py"
        dict_bases: Dicionário que armezanará os dados carregados
        regioes: lista opcional para especificar as regiões que o usuário deseja utilizar

    Returns:
        O dicionário atualizado com os Dataframes processados
        
    """
    
    for ano in sorted(os.listdir(dados_ssp_sp), reverse=True):
        dict_bases[ano] = {}
        for regiao in os.listdir(f"{dados_ssp_sp}/{ano}"):
            if regioes is None or regiao in regioes:
                dict_bases[ano][regiao] = {}
                for municipio in os.listdir(f"{dados_ssp_sp}/{ano}/{regiao}"):
                    df = leitura_municipio(ano, regiao, municipio)
                    dict_bases[ano][regiao][municipio] = df
                    
    return dict_bases

def leitura_municipio(ano, regiao, municipio):
    """
    Função auxiliar para a leitura dos arquivos .xls
    Args:
        ano: Ano referente aos crimes ocorridos
        regiao: Região do estado de SP referente aos crimes ocorridos
        municipio: Município referente aos crimes ocorridos

    Returns:
        Dataframe Pandas do município para um dado ano
    """
    dir = os.path.join(os.getcwd(), "Dados_ssp_sp", ano, regiao, municipio)
    caminho = str(dir) + "\\" + os.listdir(dir)[0]
    df = pd.read_excel(caminho)
    return df
    
def formatar_nome(crime):
    """
    Função auxiliar para formatar os tipos de crime
    
    Args:
        o tipo do crime 

    Returns:
        O nome formatado 
    """
    crime_normalizado = re.sub(r'\W+', '_', crime)
    crime_normalizado = crime_normalizado.strip('_')
    crime_normalizado = crime_normalizado.lower()
    return crime_normalizado

def corrigir_valores_coluna(valor):

    """
    Função para corrigir formatações indevidas dos dados.
    Muitos valores foram registrados como float de forma errada: 
    
    Alguns exemplos,
    1.543 -> 1543
    1.45 -> 1450
    28.000 -> 28
    1.000 -> 
    2.034 -> 2033 ????
    
    Além disso, para padronizar, transformou-se os valores "...", referentes
    a dados não registrados para 0.
    
    """    
    if isinstance(valor, str) and "..." in valor:
        return 0
    
    if isinstance(valor, (int, float)):
        valor_str = str(valor)
        
        if '.' in valor_str:
            parte_inteira, parte_decimal = valor_str.split('.')

            if parte_decimal != '0' * len(parte_decimal):
                return int(float(valor) * 1000)
            else:
                return int(parte_inteira)
    
        return int(valor)
    
    return valor

def processar_base_municipio(pre_df, nome_mun, codigos_mun, ano):
    """
    Para cada município, faz-se um dicionário no qual cada chave refere-se 
    a um crime e cada valor um dataframe que possui o número de ocorrencias
    no decorrer do ano em questão, segmentado por mês.

    Args:
        pre_df: o arquivo .xls gerado a partir do site SSP-SP, sem nenhuma alteração 
        nome_mun: o nome do município que será processado
        codigos_mun: Dataframe com os códigos e nomes dos municípios fornecido pela SEADE 
        ano: ano referente às ocorrencias criminais

    Returns:
        Dicionário no formato {'nome_do_crime': 'Ocorrencias_mensais_do_crime'}
        
    """
    df = pre_df.copy()

    # 1. Ajuste de tipos e renomeação das colunas
    for coluna in df.columns[1:]:
        df[coluna] = df[coluna].apply(corrigir_valores_coluna)    
    df.rename(columns=lambda x: f"{x} {ano}" if x in ['Janeiro', 'Fevereiro', 'Marco',
                                                       'Abril', 'Maio', 'Junho',
                                                       'Julho', 'Agosto','Setembro',
                                                       'Outubro', 'Novembro', 'Dezembro'] else x, inplace=True)
    
    # 2. Criação de um dataset para cada tipo de crime. 
    dicionario_df_crimes = {}
    for crime in df['Natureza'].unique():
        df_crime = df[df['Natureza'] == crime]

        dicionario_df_crimes[formatar_nome(crime)] = df_crime.drop(columns=['Natureza'])

    # 3. Associar as bases aos seus respectivos códigos e nomes do município
    for crime, df_crime in dicionario_df_crimes.items():
    # Merge dos dados criminais com os códigos para Junções Futuras
        df_crime['municipios'] = nome_mun
        df_crime = pd.merge(df_crime, codigos_mun, on='municipios', how='inner')

        # Reajuste da ordem das colunas
        cols = df_crime.columns.tolist()
        cols_em_primeiro = cols[-4:]
        nova_ordem_cols = cols_em_primeiro + [col for col in cols if col not in cols_em_primeiro]
        df_crime = df_crime[nova_ordem_cols]

        # Atribuindo os dataframes modificados ao dicionário
        dicionario_df_crimes[crime] = df_crime
    
    return dicionario_df_crimes
    

def concatenar_municipios(dict_novo_municipio, dict_consolidado=None):
    """
    Função para concatenar os dataframes dos crimes dos municípios.

    Args:
        dict_novo_municipio: Dicionário no formato do output da função 'processar_base_municipio()'
        dict_consolidado: Dicionário final que terá como chave o ano e como valor os dataframes
        referentes a cada crime, com todos os municípios processados, junto de seus códigos.

    Returns:
        Atualização do dict_consolidado, com o novo município concatenado
    """
    
    if dict_consolidado is None:
        dict_consolidado = {}

    for crime, df in dict_novo_municipio.items():
        if crime in dict_consolidado:
            dict_consolidado[crime] = pd.concat([dict_consolidado[crime], df], ignore_index=True)
        else:
            dict_consolidado[crime] = df
            
    return dict_consolidado

def verificar_municipios_processados(ano, regiao, dict_concatenado, dict_bases):
    """ Verificar se todos os municípios foram devidamente processados.
    """
               
    municipios_baixados = os.listdir(f"Dados_ssp_sp/{ano}/{regiao}")

    for municipio in municipios_baixados:
        for mun, crime in dict_bases[ano][regiao][municipio].items():
            # Se o primeiro crime não foi devidamente processado, 
            if crime.empty:
                print(f"O município de {municipio} não foi processado.")
                break

def verificador_corretude_dados(ano, dict_concatenado, tolerancia=1):
    """Verificação da corretude dos dados registrados após a formatação.
    """
    meses = ['Janeiro', 'Fevereiro', 'Marco', 'Abril', 'Maio', 'Junho', 
                     'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
    meses_ano = [f"{mes} {ano}" for mes in meses]

    for crime, df_crime in dict_concatenado[ano].items():
        
        df_crime[meses_ano] = df_crime[meses_ano].apply(pd.to_numeric, errors='coerce')
        df_crime['Total'] = pd.to_numeric(df_crime['Total'], errors='coerce')        
        
        df_crime['Soma_meses'] = df_crime[meses_ano].sum(axis=1)
        #df_crime['Verificado'] = df_crime['Soma_meses'] == df_crime['Total']
        df_crime['Verificado'] = (df_crime['Soma_meses'] - df_crime['Total']).abs() <= tolerancia
        
        if not df_crime['Verificado'].all():
            erros = df_crime[df_crime['Verificado'] == False]

            for index, linha in erros.iterrows():
                municipio = linha['municipios']
                print(f"Erro nos dados do municipio {municipio} no ano {ano} para o crime {crime}.")
                
        df_crime.drop(columns='Soma_meses', inplace=True)

def exportar_dataframes(dict_concatenado, caminho_destino='bases_SSP-SP_processadas', crimes_selecionados=None,
                        anos_selecionados=None):
    for ano,  crimes in dict_concatenado.items():
        if anos_selecionados is None or ano in anos_selecionados:
            for crime, df in crimes.items():
                if crimes_selecionados is None or crime in crimes_selecionados:
                    diretorio = os.path.join(caminho_destino, str(ano))
                    os.makedirs(diretorio, exist_ok=True)
                    
                    arquivo = os.path.join(diretorio, f"{crime}.csv")
                    
                    df.to_csv(arquivo, index=False)
                    
                    print(f"Exportado: {arquivo}")

In [3]:
caminho_temp = r"C:\Users\muril\Documents\2024\projeto_analise_criminal"

os.chdir(caminho_temp)
regioes = pd.read_csv("divisoes_regionais.csv")

caminho = r"C:\Users\muril\Documents\2024\projeto_analise_criminal\Processamento_base_SSP-SP"
os.chdir(caminho)

regioes = regioes[regioes['Nome_UF'] == 'São Paulo']
regioes.drop(columns=['UF', 'Nome_UF', 'Mesorregião Geográfica',
                      'Nome_Mesorregião', 'Microrregião Geográfica',
                      "Nome_Microrregião", "Município"], inplace=True)

regioes.rename(columns={'Código Município Completo': 'cod_ibge',
                        'Nome_Município': 'municipios'}, inplace=True)

codigos = pd.read_csv("codigos_municipios_regioes.csv", sep=';',
                      encoding="ISO-8859-1")
codigos.drop(columns=['cod_ra', 'cod_ibge6', 'pop2020', 'porte_pop2020', 'cod_drs', 'drs'],
             axis=1, inplace=True)
codigos = codigos.iloc[1:, :]

codigos = codigos.merge(regioes, on=['cod_ibge'], how='inner')
codigos = codigos.drop_duplicates().drop(columns='municipios_y').rename(columns={'municipios_x': 'municipios'})

In [4]:
# Pela função verificar_municipios_processados, sabe-se que os municípios de Mogi Mirim e Santo Antônio de Posse
# possuem nomes que divergem
nomes_divergentes = ['Moji Mirim', 'Santo Antonio de Posse', 'Arco Íris', 'Embu-Guaçu',
                     'Santo Antonio do Jardim', 'Santa Lucia', 'Santo Antonio da Alegria',
                     'Santo Antonio do Pinhal', 'Santo Antonio do Aracanguá']

nomes_corretos = ['Mogi Mirim', 'Santo Antônio de Posse', 'Arco-Íris', 'Embu Guaçú',
                  'Santo Antônio do Jardim', 'Santa Lúcia', 'Santo Antônio da Alegria',
                  'Santo Antônio do Pinhal', 'Santo Antônio do Aracanguá']

mapa_nomes = dict(zip(nomes_divergentes, nomes_corretos))

codigos['municipios'] = codigos['municipios'].replace(mapa_nomes)

In [5]:
from tqdm import tqdm

dict_bases = {}
dict_concatenado = {}
regioes_baixadas = ['Araçatuba', 'Bauru', 'Campinas', 'Capital', 'Grande São Paulo (exclui a Capital)',
        'Piracicaba', 'Presidente Prudente', 'Ribeirão Preto', 'Santos', 'São José do Rio Preto',
        'São José dos Campos', 'Sorocaba']

# Chama a função para carregar as bases
leitura_bases('Dados_ssp_sp', dict_bases, regioes_baixadas)

# Adiciona barra de progresso para o loop principal de anos
for ano, regioes in tqdm(dict_bases.items(), desc="Processando anos"):
    dict_concatenado[ano] = {}
    
    # Adiciona barra de progresso para o loop das regiões
    for regiao, municipios in tqdm(regioes.items(), desc=f"Processando regiões ({ano})", leave=False):
        
        # Adiciona barra de progresso para o loop dos municípios
        for municipio, df in tqdm(municipios.items(), desc=f"Processando municípios ({regiao})", leave=False):
            dict_bases[ano][regiao][municipio] = processar_base_municipio(df, municipio, codigos, ano)
            concatenar_municipios(dict_bases[ano][regiao][municipio], dict_concatenado[ano])


Processando anos:   0%|          | 0/15 [00:00<?, ?it/s]

























































































































































































































































































































































































































































Processando anos:   7%|▋         | 1/15 [01:03<14:45, 63.26s/it]


































































































































































































































































































































































































Processando anos:  13%|█▎        | 2/15 [01:59<12:4

In [ ]:
anos = ['2010', '2011', '2012', '2013', '2014', '2015','2016', '2017', '2018', '2019',
        '2020', '2021', '2022', '2023', '2024']

for regiao in regioes_baixadas:
    verificar_municipios_processados('2024', regiao, dict_concatenado, dict_bases)


for ano in anos:
    verificador_corretude_dados(ano, dict_concatenado, tolerancia=10)


Erro nos dados do municipio Itaquaquecetuba no ano 2010 para o crime total_de_roubo_outros_1.
Erro nos dados do municipio Itatiba no ano 2012 para o crime furto_outros.
Erro nos dados do municipio Jundiaí no ano 2016 para o crime lesão_corporal_dolosa.
Erro nos dados do municipio Americana no ano 2017 para o crime furto_outros.
Erro nos dados do municipio Pindamonhangaba no ano 2020 para o crime furto_outros.
Erro nos dados do municipio Carapicuíba no ano 2023 para o crime furto_outros.


In [27]:
# Processamento ano de 2015
# Os dados de Estupro que não foram registrados podem ser calculados a partir da variável "total_de_estupro_(4)"
# Roubo outros: Sofre de um processo semelhante aos Dados de estupro
furto_outros_2015 = dict_concatenado['2015']['furto_outros']

for coluna in furto_outros_2015.iloc[:, 4:16]:
    furto_outros_2015[coluna] = furto_outros_2015[coluna].apply(corrigir_valores_coluna)

dict_concatenado['2015']['furto_outros'] = furto_outros_2015

In [38]:
# Após verificar a corretude de todos os anos, deve-se, manualmente alterar cada uma das observações significantes
# 
# 2015 - Os tipos de crime "estupro" e "roubo_outros" não puderam ser processados
# Tanto os casos de "Jundiaí 2016 lesao_corporal_dolosa"  e "Itatiba 2012 furto_outros"
# Sofrem do mesmo erro 

dict_concatenado['2012']['furto_outros'].loc[dict_concatenado['2012']['furto_outros']['municipios'] == 'Itatiba', 'Total'] = 1000
dict_concatenado['2016']['lesão_corporal_dolosa'].loc[dict_concatenado['2016']['lesão_corporal_dolosa']['municipios'] == 'Jundiaí', 'Total'] = 1000
dict_concatenado['2020']['furto_outros'].loc[dict_concatenado['2020']['furto_outros']['municipios'] == 'Pindamonhangaba', 'Total'] = 1000
dict_concatenado['2010']['total_de_roubo_outros_1'].loc[dict_concatenado['2010']['total_de_roubo_outros_1']['municipios'] == 'Itaquaquecetuba', 'Total'] = 1000
dict_concatenado['2017']['furto_outros'].loc[dict_concatenado['2017']['furto_outros']['municipios'] == 'Americana', 'Total'] = 3000
dict_concatenado['2023']['furto_outros'].loc[dict_concatenado['2023']['furto_outros']['municipios'] == 'Carapicuíba', 'Total'] = 3000



In [40]:
exportar_dataframes(dict_concatenado, caminho_destino="regioes(2010, a 2024)",
                    anos_selecionados=['2010', '2011', '2012', '2013', '2014', '2015','2016', '2017', '2018', '2019',
        '2020', '2021', '2022', '2023', '2024'],
                    crimes_selecionados=['nº_de_vítimas_em_homicídio_doloso_3',
                                         'homicídio_culposo_por_acidente_de_trânsito',
                                         "homicídio_culposo_outros",
                                         "tentativa_de_homicídio",
                                         "lesão_corporal_dolosa",
                                         "lesão_corporal_culposa_por_acidente_de_trânsito",
                                         "lesão_corporal_culposa_outras",
                                         "latrocínio",
                                         "total_de_estupro_4",
                                         "total_de_roubo_outros_1",
                                         "furto_outros",
                                         "furto_de_veículo"])

Exportado: regioes(2010, a 2024)\2024\nº_de_vítimas_em_homicídio_doloso_3.csv
Exportado: regioes(2010, a 2024)\2024\homicídio_culposo_por_acidente_de_trânsito.csv
Exportado: regioes(2010, a 2024)\2024\homicídio_culposo_outros.csv
Exportado: regioes(2010, a 2024)\2024\tentativa_de_homicídio.csv
Exportado: regioes(2010, a 2024)\2024\lesão_corporal_dolosa.csv
Exportado: regioes(2010, a 2024)\2024\lesão_corporal_culposa_por_acidente_de_trânsito.csv
Exportado: regioes(2010, a 2024)\2024\lesão_corporal_culposa_outras.csv
Exportado: regioes(2010, a 2024)\2024\latrocínio.csv
Exportado: regioes(2010, a 2024)\2024\total_de_estupro_4.csv
Exportado: regioes(2010, a 2024)\2024\total_de_roubo_outros_1.csv
Exportado: regioes(2010, a 2024)\2024\furto_outros.csv
Exportado: regioes(2010, a 2024)\2024\furto_de_veículo.csv
Exportado: regioes(2010, a 2024)\2023\nº_de_vítimas_em_homicídio_doloso_3.csv
Exportado: regioes(2010, a 2024)\2023\homicídio_culposo_por_acidente_de_trânsito.csv
Exportado: regioes(201